In [13]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

In [14]:
MU = 3.986*10**5#地球重力定数[km^3/s^2]
R_po = 6600.#パーキング軌道半径[km]
Delta_i = 30.#種子島の緯度[deg]

In [41]:
class Pattern:
    """なんか軌道わかるとうまくいくやつ()"""
    def __init__(self, r_tgt):
        """とりあえず円周回軌道半径だけ。。。"""
        self.r_tgt = r_tgt
        self.delta_v_peri = None
        self.delta_v_apo = None
        self.delta_v = None
        
    def get_delta_v_peri(self):
        """近地点でのΔv"""
        r_tgt = self.r_tgt
        v_po = np.sqrt(MU/R_po)
        v_peri_trans = np.sqrt(2*MU*r_tgt/R_po/(R_po+r_tgt))
        self.delta_v_peri = v_peri_trans - v_po

    def get_delta_v_apo(self):
        """遠地点でのΔv"""
        r_tgt = self.r_tgt
        v_tgt = np.sqrt(MU/r_tgt)
        v_apo_trans = np.sqrt(2*MU*R_po/r_tgt/(R_po+r_tgt))
        self.delta_v_apo = np.sqrt(v_tgt**2+v_apo_trans**2-2*v_tgt*v_apo_trans*np.cos(Delta_i/180.*np.pi))
    
    def get_delta_v(self):
        """Δv全部"""
        self.get_delta_v_peri()
        self.get_delta_v_apo()
        self.delta_v = self.delta_v_apo+self.delta_v_peri
        print(self.delta_v)

In [43]:
pattern1 = Pattern(7700.)